# DBsimilarity — Basics in Chemometrics

Computational methods are now core to natural products research, especially when dealing with **unfractionated extracts** where signal overlap and chemical diversity complicate classic workflows. **Chemoinformatics** helps turn raw structures into actionable knowledge for:

* specimen prioritization,
* hypothesis generation for biological activity,
* biosynthetic and taxonomic inference,
* and rapid *in silico* dereplication.

## Goal of this notebook

Provide a clear, reproducible, end-to-end path that turns structure files into analysis-ready artifacts for MS/NMR dereplication and chemical space exploration—without requiring advanced programming skills.

## What you’ll do (overview)

1. **Convert** structural libraries (e.g., `.sdf`) to tidy **`.csv`** tables.
2. **Organize** data into a single Pandas **DataFrame** with consistent identifiers.
3. **Annotate** each SMILES with chemoinformatics fields (InChI, InChIKey, formula, exact mass, adduct m/z, SLogP).
4. **Build a custom MS database** (MZmine-style) for rapid **MS1 dereplication**.
5. **Assemble candidate lists for 2D NMR** dereplication (e.g., to support HSQC/TOCSY workflows).
6. **Quantify similarity** between compounds (Morgan fingerprints) and **cluster** using descriptors.
7. **Construct similarity networks** and export edge lists ready for **Cytoscape**.

Along the way you’ll also generate **MassQL** queries (MS1 and MS2) to search vendor-agnostic LC-MS(/MS) datasets, compute **Mordred** descriptors, run **hierarchical clustering** and **t-SNE** projections to visualize chemical space.

## Who this is for

Researchers who want practical, readable notebooks to:

* understand what’s in a specimen-specific compound list,
* create reusable MS/NMR dereplication assets,
* and explore chemical diversity with minimal coding.

## Inputs & outputs (at a glance)

* **Input:** structure files (`.sdf`) or tables (`.csv`) with at least names and SMILES.
* **Outputs:**

  * Annotated compound table (InChI, InChIKey, formula, exact mass, adduct m/z, SLogP)
  * MZmine custom DB CSV (MS1)
  * MassQL query files (MS1 and MS2)
  * Descriptor tables (Mordred) + dendrogram PNG + clustering CSV
  * t-SNE HTML plot (interactive)
  * Similarity network edge list + isolated nodes CSV (for Cytoscape)

## Minimal prerequisites

* **Python** (conda environment recommended)
* Core libraries: `pandas`, `numpy`, `matplotlib`, `scipy`, `scikit-learn`, `networkx`
* **RDKit** for structure parsing and fingerprints
* **Mordred** (optional) for extended descriptors
* **Plotly** (optional) for interactive t-SNE
* **Cytoscape** (optional) for network visualization

> Tip: Some RDKit/Mordred builds prefer `numpy<2`. If descriptor steps fail, pin NumPy accordingly.

## Learn more

* Pandas: [https://pandas.pydata.org/](https://pandas.pydata.org/)
* RDKit (Getting Started in Python): [https://www.rdkit.org/docs/GettingStartedInPython.html](https://www.rdkit.org/docs/GettingStartedInPython.html)
* ChEMBL web services: [https://chembl.gitbook.io/chembl-interface-documentation/web-services](https://chembl.gitbook.io/chembl-interface-documentation/web-services)
* Mordred: [http://mordred-descriptor.github.io/documentation/master/](http://mordred-descriptor.github.io/documentation/master/)


In [1]:
# --- Toggles (flip as you like) ---
USE_NBAGG          = True   # interactive matplotlib in Jupyter (fallbacks if unsupported)
ENABLE_PANDAS_TOOLS= True   # RDKit PandasTools (adds Molecule column helpers)
ENABLE_MORDRED     = True   # set False if not installed
ENABLE_CHEMBL      = False  # set True if you plan to query ChEMBL

# --- Stdlib & core libs ---
import os, sys, warnings, platform
from pathlib import Path
import numpy as np
import pandas as pd

# Pandas display niceties
pd.set_option("display.max_rows", 50)
pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 180)
pd.set_option("display.float_format", lambda x: f"{x:.6g}")

# --- RDKit (don’t alias AllChem as Chem!) ---
try:
    from rdkit import RDLogger, Chem
    RDLogger.DisableLog('rdApp.*')          # silence verbose RDKit logs
    from rdkit.Chem import AllChem          # fingerprints, conformers
    from rdkit.Chem import Descriptors as RDDesc
    from rdkit.Chem.rdMolDescriptors import CalcMolFormula
    from rdkit import DataStructs
    RDKIT_OK = True
except Exception as e:
    RDKIT_OK = False
    print("⚠️ RDKit not available:", e)

# RDKit PandasTools (optional, controlled by toggle)
HAS_PANDAS_TOOLS = False
if ENABLE_PANDAS_TOOLS and RDKIT_OK:
    try:
        from rdkit.Chem import PandasTools
        HAS_PANDAS_TOOLS = True
    except Exception as e:
        print("ℹ️ RDKit PandasTools not available (continuing without it):", e)

# --- Mordred (optional) ---
if ENABLE_MORDRED:
    try:
        from mordred import Calculator, descriptors
        MORDRED_OK = True
    except Exception as e:
        MORDRED_OK = False
        print("⚠️ Mordred not available:", e)
else:
    MORDRED_OK = False

# --- Plotting ---
import matplotlib
if USE_NBAGG:
    try:
        matplotlib.use('nbagg')  # interactive zoom when supported
    except Exception:
        pass
import matplotlib.pyplot as plt
try:
    import seaborn as sns
    sns.set_context("notebook")
    sns.set_style("ticks")
except Exception:
    pass

# --- Plotly (optional, for interactive HTML like t-SNE) ---
try:
    import plotly
    PLOTLY_OK = True
except Exception as e:
    PLOTLY_OK = False
    print("ℹ️ Plotly not available (only affects interactive HTML exports):", e)

# --- Networks & misc ---
import networkx as nx
try:
    from wordcloud import WordCloud
except Exception:
    WordCloud = None

# --- ChEMBL client (optional) ---
if ENABLE_CHEMBL:
    try:
        from chembl_webresource_client.new_client import new_client
    except Exception as e:
        print("ℹ️ ChEMBL client not available (skipping):", e)
        new_client = None
else:
    new_client = None

# --- Project paths ---
PROJECT_ROOT = Path.cwd()
DATA_DIR     = PROJECT_ROOT / "data"
IMAGES_DIR   = PROJECT_ROOT / "images"
DATA_DIR.mkdir(exist_ok=True)
IMAGES_DIR.mkdir(exist_ok=True)

# --- Import your tools ---
# Prefer local project modules; fall back to the generic chem tools if present.
sys.path.append(str(PROJECT_ROOT))
sys.path.append(str(PROJECT_ROOT.parent))

dbs = None
ct  = None
try:
    sys.path.insert(0, os.path.abspath('..'))
    import dbsimilarity_2 as dbs
    print("✅ Loaded dbsimilarity_2 as dbs")
except Exception as e:
    print("ℹ️ Could not import dbsimilarity_2:", e)


aliases = {
    "SMILES":   ["SMILES", "smiles", "***smiles***", "Smiles", "Smiles_parent"],
    "InChIKey": ["InChIKey", "INCHIKEY", "inchikey", "InChlKey", "InChI Key", "INCHI_KEY"],
    "Compound name": ["Compound name", "Name", "Compound", "compound_name"],
}        

# --- Quick environment summary ---
def env_summary():
    print("Python :", platform.python_version())
    print("Pandas :", pd.__version__)
    print("NumPy  :", np.__version__)
    if RDKIT_OK:
        try:
            from rdkit.rdBase import rdkitVersion
            print("RDKit  :", rdkitVersion)
        except Exception:
            print("RDKit  : OK (version not available)")
    else:
        print("RDKit  :", "missing")
    print("Mordred:", "OK" if MORDRED_OK else "missing/disabled")
    print("Plotly :", "OK" if PLOTLY_OK else "missing/disabled")
    print("PandasTools:", "OK" if HAS_PANDAS_TOOLS else "missing/disabled")

env_summary()

✅ Loaded dbsimilarity_2 as dbs
Python : 3.11.6
Pandas : 1.5.3
NumPy  : 1.23.5
RDKit  : 2022.09.5
Mordred: OK
Plotly : OK
PandasTools: OK


### Set the project name from the folder you’re working in

In [2]:
# Get the current working directory
current_directory = %pwd

# Extract the last folder from the path
last_folder = os.path.basename(current_directory)
Project = last_folder

Project

'Cyanobacteria'

### Load the target compound list and normalize identifiers

This cell reads your **target list** from `Target+".csv"` (semicolon-separated) and derives a canonical **InChIKey** from each SMILES. Having InChIKeys up front makes downstream matching, deduplication, and network building reliable and ID-agnostic.

Tips:

* Make sure the CSV has a **`SMILES`** column (and optionally a name/ID column).
* If your file uses a different delimiter or encoding, tweak `sep=";"` or add `encoding="utf-8"`.
* Invalid SMILES will raise errors later; consider filtering or reporting them here if needed.
* If you already have an `InchiKey` column, you can skip recomputing—or recompute to ensure consistency.


In [3]:
Target = "CyanoMetDB"  # pass either a stem or filename with extension

path = dbs._detect_file(Target)
sep = dbs._guess_sep(path)
df_target, used_encoding = dbs._read_table(path, sep=sep)
smiles_col = dbs._normalize_smiles_column(df_target)

df_target = dbs.rename_by_aliases(df_target, aliases, inplace=False, prefer="max_notna")

dbs._add_inchikey(df_target, smiles_col="SMILES", out_col="InchiKey")

print(
    f"Loaded: {path.name} | sep='{sep}' | encoding='{used_encoding}' | "
    f"rows={df_target.shape[0]}, cols={df_target.shape[1]}"
)
invalid = df_target["InchiKey"].isna().sum()
if invalid:
    print(f"Note: {invalid} rows have invalid/unparseable SMILES (InchiKey = NaN).")

df_target.head(3)

Loaded: CyanoMetDB.csv | sep=',' | encoding='utf-8' | rows=2122, cols=6
Note: 7 rows have invalid/unparseable SMILES (InchiKey = NaN).


,Compound identifier,Compound name,SMILES,InChIKey,Fragments,InchiKey
0,CyanoMetDB_0002,Anhydrohapaloxindole B,O=C1NC2=CC=CC3=C2C1=C4[C@H]([C@@](C=C)([C@@H](...,POWOOZMDXKYYOK-FWKFCYAVSA-N,NaN,POWOOZMDXKYYOK-FWKFCYAVSA-N
1,CyanoMetDB_0003,Columbamide B,ClC(Cl)CCCCCC(Cl)CCCC/C=C/CCC(N(C)[C@@H](COC)C...,JBEYLLUXESVNSE-QOZDHKFNSA-N,NaN,JBEYLLUXESVNSE-QOZDHKFNSA-N
2,CyanoMetDB_0004,Columbamide C,ClCCCCCCC(Cl)CCCC/C=C/CCC(N(C)[C@@H](COC)CO)=O,HFTYBOHUADLAJY-KRTLOWCSSA-N,NaN,HFTYBOHUADLAJY-KRTLOWCSSA-N


### Load the reference (“compounds of interest”) database and standardize IDs

This cell imports your curated reference set (e.g., CyanoMetDB / ChEMBL slice) from a semicolon-separated CSV encoded in **Latin-1**. It also fixes a common header typo by renaming **`InChlKey` → `InchiKey`**, so downstream joins and comparisons use a single, consistent identifier.

Why it matters:

* This table is your **search space** for dereplication and similarity checks against the target list.
* Consistent **InchiKey** naming avoids silent merge failures later.

Tips:

* After loading, consider trimming whitespace and **deduplicating by `InchiKey`** to prevent double counting.
* If you see weird characters, confirm the file’s encoding (utf-8 vs latin1) and delimiter (`sep=";"` here).
* Keep column names aligned across tables (`SMILES`, `InchiKey`, `Compound name`, etc.) for smooth merges.


In [4]:
Additional = "CyanoNP2010-2023"  # pass either a stem or filename with extension

path = dbs._detect_file(Additional)
sep = dbs._guess_sep(path)
df_Additional, used_encoding = dbs._read_table(path, sep=sep)
smiles_col = dbs._normalize_smiles_column(df_Additional)

df_Additional = dbs.rename_by_aliases(df_Additional, aliases, inplace=False, prefer="max_notna")

dbs._add_inchikey(df_Additional, smiles_col="SMILES", out_col="InchiKey")

print(
    f"Loaded: {path.name} | sep='{sep}' | encoding='{used_encoding}' | "
    f"rows={df_Additional.shape[0]}, cols={df_Additional.shape[1]}"
)
invalid = df_Additional["InchiKey"].isna().sum()
if invalid:
    print(f"Note: {invalid} rows have invalid/unparseable SMILES (InchiKey = NaN).")

df_Additional.head(3)

Loaded: CyanoNP2010-2023.csv | sep=',' | encoding='utf-8' | rows=995, cols=4


,file_path,SMILES,Fragments,InchiKey
0,4-hydroperoxyoscillatoxin B2,O=C(O[C@@H]([C@H](O)C)CC(O[C@H]([C@H](C)[C@H](...,NaN,QZGGIZIUEKEAAS-XRPLRUMHSA-N
1,17-bromo-4-hydroperoxyoscillatoxin B2,O=C(O[C@@H]([C@H](O)C)CC(O[C@H]([C@H](C)[C@H](...,NaN,IXKNJBBLNNUXPK-KASRWXAJSA-N
2,17-bromooscillatoxin B2,O=C(O[C@@H]([C@H](O)C)CC(O[C@H]([C@H](C)[C@H](...,NaN,YDMZOYPZIGUJJL-KASRWXAJSA-N


#### Clean and deduplicate the reference table

This step removes unusable entries and redundancies before any joins or similarity work:

* Converts empty `SMILES` strings to `NaN` and **drops missing** structures.
* **Deduplicates** by `SMILES` to avoid double counting during stats, clustering, and network building.
* Prints the table size **before/after** so you can audit the impact.

Tips:

* If tautomers/salts matter, consider normalizing SMILES (`Chem.MolToSmiles(..., canonical=True)`) or deduplicating by **InChIKey** instead of raw SMILES.
* Optional: trim whitespace and standardize encoding before this step to catch edge cases.


In [5]:
# Filter row for empty cells
print(f"Size of the dataframe before filter empty cells: {df_Additional.shape}.")
df_Additional["SMILES"].replace('',np.nan,inplace=True)
df_Additional.dropna(subset="SMILES", inplace=True)
# Remove duplicate rows based on the "SMILES" column
df_Additional.drop_duplicates(subset=["SMILES"], inplace=True)
print(f"Size of the dataframe after filter empty and duplicated cells: {df_Additional.shape}.")

Size of the dataframe before filter empty cells: (995, 4).
Size of the dataframe after filter empty and duplicated cells: (995, 4).


### Merge target and reference, then flag the overlap (“compounds of interest”)

This cell unions the **target** list and the **reference** database on `InchiKey` using `merge_dataframes()`. That helper also adds presence flags:

* `additional_column_1` → present in **df\_target**
* `additional_column_2` → present in **df\_Additional_List**

We then extract the **intersection**—compounds that occur in **both** tables—as `df_Interest`, and print a quick audit of sizes so you can see how many candidates are immediately dereplicated.

Heads-up:

* Use parentheses when filtering to avoid precedence gotchas:

  ```python
  df_Interest = merged_df.loc[(merged_df['additional_column_1'] == 1) & (merged_df['additional_column_2'] == 1)]
  ```
* Consider renaming the flags after merge (e.g., `in_target`, `in_reference`) for readability.
* If stereochemistry/salt forms matter, `InchiKey` is a good join key; otherwise you may want to use standardized InChI or layer-specific keys depending on your workflow.


In [6]:
# Combine both dataframes
merged_df = dbs.merge_dataframes(
    [df_target,df_Additional],
    key_column="InchiKey",
    name_columns=["Compound name", "file_path", "Compound"],
    unified_name_col="Compound name (unified)"
)

df_Interest = merged_df.loc[merged_df['additional_column_1'] & merged_df['additional_column_2'] == 1]
print(f"Size of the merged structure databases: df_target + df_Additional ({df_target.shape[0]+df_Additional.shape[0]}) = {merged_df.shape[0]}")
print(f"Size of the original structure database to study: {df_target.shape[0]}")
print(f"Size of the Additional structure database used as seed for compounds of interest: {df_Additional.shape[0]}")
print(f"How many compounds can be promptly identified as compounds of interest (co-occurring in both databases): {df_Interest.shape[0]}")

Size of the merged structure databases: df_target + df_Additional (3117) = 2709
Size of the original structure database to study: 2122
Size of the Additional structure database used as seed for compounds of interest: 995
How many compounds can be promptly identified as compounds of interest (co-occurring in both databases): 383


### Find MS2 Fragments from .mgf (GNPS)

In [7]:
from pathlib import Path

# resolve ../resources relative to the current working directory
RESOURCES_DIR = (Path.cwd().parent / "resources").resolve()
print("Looking in:", RESOURCES_DIR)

spectra_by_batch = dbs.load_mgf_spectra(str(RESOURCES_DIR))
print("Batches loaded:", list(spectra_by_batch.keys())[:5])

# Keeps up to 10 most intense fragment peaks per spectrum (after thresholding)
# Only peaks with relative intensity ≥ 2% of the base peak are considered
mgfdata_df = (
    dbs.spectra_to_dataframe(
        spectra_by_batch,
        top_n=5,       # X: max number of fragment peaks to keep per spectrum
        min_rel_pct=20.0 # Y: minimum relative intensity (%) vs. base peak
    )
    .rename(columns={"fragments": "Fragments"})  # optional, just cosmetic
)

display(mgfdata_df.head())

# GraphML → DataFrame
df_nodes = dbs.graphml_to_dataframe(
    "Library-b0b1079fccb74a26aeddc992605f19bf-network_singletons.graphml",
    resources_dir=RESOURCES_DIR,
    write_csv=True)  # optional

print("Nodes loaded:", len(df_nodes))
df_nodes = dbs.add_inchikey_from_structures(df_nodes[["id", "SMILES"]])
display(df_nodes.head())

Looking in: C:\Users\borge\Documents\DBsimilarity.py\resources
Batches loaded: ['Library-b0b1079fccb74a26aeddc992605f19bf-specs_ms']


,batch,scans,scan_number,precursor_mass,Fragments,n_fragments
0,Library-b0b1079fccb74a26aeddc992605f19bf-specs_ms,1,1,393.207,147.0802:100.0;171.0802:40.5;121.0645:40.5;107...,5
1,Library-b0b1079fccb74a26aeddc992605f19bf-specs_ms,2,2,393.208,147.0802:100.0;355.1909:73.2;237.1273:70.6;337...,5
2,Library-b0b1079fccb74a26aeddc992605f19bf-specs_ms,3,3,393.208,147.0802:100.0;355.1911:64.0;237.1274:63.8;171...,5
3,Library-b0b1079fccb74a26aeddc992605f19bf-specs_ms,4,4,393.208,147.0803:100.0;237.1274:64.9;355.1912:62.7;171...,5
4,Library-b0b1079fccb74a26aeddc992605f19bf-specs_ms,5,5,393.208,147.0802:100.0;237.1274:65.0;355.1912:62.3;171...,5


Nodes loaded: 259621


,id,SMILES,InchiKey
0,94850,CC(CN1CCOCC1)OC(=O)c1ccc(C(C)(C)C)cc1,BEXIDRCZUGWGPI-UHFFFAOYSA-N
1,103461,CC(=O)OC1(C)CC(C)C(=O)C(C(O)CC2CC(=O)NC(=O)C2)C1,UFDHNJJHPSGMFX-UHFFFAOYSA-N
2,222897,CN(C)CCOC(=O)COc1ccc(Cl)cc1,XZTYGFHCIAKPGJ-UHFFFAOYSA-N
3,144738,Cc1ccc(-c2c(C(=O)N3CCN4C(=O)NC(=O)C4C3)scc2)cc1,HGPQJSIALUOWRP-UHFFFAOYSA-N
4,152314,NC(=O)c1c(F)ccc(NC(=O)c2ccccc2OC(F)(F)F)c1,IOOZNCLKVQZBGI-UHFFFAOYSA-N


In [8]:
display(mgfdata_df.tail())

,batch,scans,scan_number,precursor_mass,Fragments,n_fragments
259616,Library-b0b1079fccb74a26aeddc992605f19bf-specs_ms,259617,259617,852.802,579.5811:100.0;551.5564:69.4,2
259617,Library-b0b1079fccb74a26aeddc992605f19bf-specs_ms,259618,259618,852.802,579.5335:100.0;551.5022:62.0,2
259618,Library-b0b1079fccb74a26aeddc992605f19bf-specs_ms,259619,259619,956.865,655.5887:100.0;607.5724:50.5,2
259619,Library-b0b1079fccb74a26aeddc992605f19bf-specs_ms,259620,259620,956.865,655.5648:100.0;607.5649:66.9;95.0854:35.8;315....,5
259620,Library-b0b1079fccb74a26aeddc992605f19bf-specs_ms,259621,259621,936.896,663.6571:100.0;635.6091:79.0;607.5424:37.8;579...,5


In [9]:
merged_df_plus_nodes = dbs.merge_on_inchikey(merged_df, df_nodes, how="left")
display(merged_df_plus_nodes.head())

summary, left_dups, right_dups, miss_left, miss_right = dbs.inchikey_match_report(
    merged_df, df_nodes, left_name="merged_df", right_name="df_nodes")

# count rows that actually matched something from df_nodes
display(summary)

,Compound identifier,Compound name,SMILES,Fragments,file_path,Compound name (unified),additional_column_1,additional_column_2,InchiKey,id
0,NaN,NaN,C(CC/C=C/C=C/[C@@H]1C([C@@H](C[C@H](C[C@H](C[C...,NaN,akunolide D,akunolide D,0,1,AALRUTPHSVMTFN-OGKHLYSKSA-N,NaN
1,NaN,NaN,CCCCCC(O)/C=C/CCCCCCCC(OCCO)=O,NaN,2-hydroxyethyl-11-hydroxyhexadec-9-enoate,2-hydroxyethyl-11-hydroxyhexadec-9-enoate,0,1,AAXLHCBKRIITLB-JLHYYAGUSA-N,NaN
2,CyanoMetDB_1644,Mirabazole C,CC(C1=N[C@](C2=N[C@@](C3=N[C@@](C4=NC=CS4)(C)C...,NaN,NaN,Mirabazole C,1,0,AAZNLXXTBVTBKQ-KSZLIROESA-N,NaN
3,CyanoMetDB_1430,Malyngamide T,CCCCCCC[C@@H](C/C=C/CCC(=O)NC/C(=C/Cl)/CC1=CC(...,NaN,NaN,Malyngamide T,1,0,ABBPFXQJIWUCKF-CXRCMLCDSA-N,NaN
4,CyanoMetDB_1700,"2,5-dimethyldodecanoic acid",CCCCCCCC(C)CC[C@@H](C)C(=O)O,NaN,NaN,"2,5-dimethyldodecanoic acid",1,0,ABQDDNHUZOOWMZ-ZGTCLIOFSA-N,NaN


,metric,value
0,merged_df rows,2709
1,df_nodes rows,259621
2,merged_df unique InchiKey,2688
3,df_nodes unique InchiKey,29993
4,unique matches (intersection),1
5,unique union,32680
6,row-level matches in merged_df,1
7,entries merged (left join merged_df→df_nodes),1
8,unique match rate vs merged_df,0.000372024
9,unique match rate vs df_nodes,3.33411e-05


In [10]:
# Now to populate the Fragments column
merged_final_frag, report = dbs.merge_nodes_with_mgf(
    merged_df_plus_nodes,
    mgfdata_df,
    dedupe_mgf=True,
    prefer_mgf_fragments=True,
    drop_nan_fragments=True,   # set True if you want to remove rows without Fragments
)
display(report)
merged_final_frag[["InchiKey","precursor_mass", "scan_number", "Compound name (unified)", "Fragments"]].to_csv("merged_fragments.csv", sep=";")

{'left_rows': 2713,
 'right_rows': 259621,
 'unique_ids': 5,
 'unique_scans': 259621,
 'matched_rows': 5,
 'match_rate_rows': 0.0018429782528566164,
 'unmatched_id_examples': []}

In [11]:
target_name = "your compound name"

mask = (
    merged_df_plus_nodes["Compound name (unified)"]
      .astype("string").str.strip().str.casefold()
    == target_name.strip().casefold()
)
rows = merged_df_plus_nodes.loc[mask]

# first row only (if you expect one)
row = rows.iloc[0] if not rows.empty else None

In [12]:
STOP

NameError: name 'STOP' is not defined

### Annotate structures with formula, exact mass, adduct m/z, InChI/InChIKey, and SLogP (customizable)

This cell runs the all-in-one annotator on `merged_df` and writes an extended table to `CyanosNP2010-2023_extended.csv`. You can toggle each computed field and choose data sources (Mordred vs RDKit for logP), ion mass convention (proton vs hydrogen atom), numeric precision, and which **adduct columns** to emit.

What you’ll get added (when enabled):

* `MolFormula`, `MolWeight` (neutral exact mass)
* Adduct m/z columns: `MolWeight-1H`, `MolWeight+1H`, `MolWeight+Na`, `MolWeight+K` (and optional `MolWeight+NH4`)
* `Inchi`, `InchiKey`
* `SLogP`

Notes & tips:

* `logp_source="mordred"` falls back to RDKit if Mordred isn’t available.
* `use_proton_mass=True` uses 1.007276466 for H⁺ / H⁻ (MS-friendly). Set `False` to use 1.007825032 (H atom).
* `drop_invalid_smiles=True` removes rows that can’t be parsed by RDKit (keeps outputs clean).
* Adjust `decimals` if you need more/less precision for m/z matching.


In [ ]:
# Fully customized run
merged_df2 = dbs.annotate_smiles_table(
    merged_df,
    smiles_col="SMILES",
    compute_inchi=True,
    compute_inchikey=True,
    compute_formula=True,
    compute_exact_mass=True,
    compute_adduct_columns=True,
    compute_logp=True,
    logp_source="mordred",  # or "rdkit"
    # Your column names → adducts (you can add/remove as you like)
    adduct_columns={
        "MolWeight-1H": "[M-H]-",
        "MolWeight+1H": "[M+H]+",
        "MolWeight+Na": "[M+Na]+",
        "MolWeight+K":  "[M+K]+",
        # "MolWeight+NH4": "[M+NH4]+",    # uncomment if desired
    },
    use_proton_mass=True,    # False -> use hydrogen atom mass (1.007825032)
    decimals=6,
    drop_invalid_smiles=True,
    out_csv=f"{Project}_extended.csv")

### Generate MS1 MassQL queries (one per compound) and save to a text file

This cell builds **vendor-agnostic MassQL** queries that search MS1 spectra for each compound’s expected **adduct m/z** values (here: `[M+H]+`, `[M+Na]+`, `[M+K]+`, `[M+NH4]+`) within a **±ppm** tolerance and with a minimum **intensity percent**. It writes a numbered, human-readable list to `MS1_queries_by_compound.txt`, with one block per compound.

Notes & tips:

* `mass_col="MolWeight"` should be the **neutral monoisotopic mass**; adduct m/z are computed from it.
* Tune `ppm`, `intensity_percent`, and `decimals` to match your instrument and centroiding.
* Set `separate_adducts=True` if you prefer **one query per adduct** instead of an OR list.
* If names aren’t unique, consider prefixing with an internal ID to avoid ambiguity in the output.


In [ ]:
# Build queries
q_dict = dbs.generate_massql_queries(
    merged_df2,
    ppm=10,
    intensity_percent=1,
    decimals=5,
    separate_adducts=False,
    adducts=["[M+H]+", "[M+Na]+", "[M+K]+", "[M+NH4]+", "[M+2H]+2"],
    name_col="Compound name",
    mass_col="MolWeight",)

out_path = Path("MS1_queries_by_compound.txt")
with out_path.open("w", encoding="utf-8") as f:
    for i, (name, q) in enumerate(q_dict.items(), start=1):
        f.write(f"### {i}. {name} ###\n{q}\n\n")  # blank line between queries


In [ ]:
# Build queries
q_dict = dbs.generate_massqlPrec_queries(
    merged_df2,
    ppm=10,
    intensity_percent=1,
    decimals=5,
    separate_adducts=False,
    adducts=["[M+H]+", "[M+Na]+", "[M+K]+", "[M+NH4]+", "[M+2H]+2"],
    name_col="Compound name",
    mass_col="MolWeight",)

out_path = Path("MS1_queries_by_compound_Prec.txt")
with out_path.open("w", encoding="utf-8") as f:
    for i, (name, q) in enumerate(q_dict.items(), start=1):
        f.write(f"### {i}. {name} ###\n{q}\n\n")  # blank line between queries

### Generate MS2 MassQL queries (precursor ± fragments) and save to a text file

This cell creates **MS2** queries that combine:

* a **precursor** constraint built from the neutral mass and the selected adducts (`[M+H]+`, `[M+Na]+`, `[M+K]+`, `[M+NH4]+`) with `ppm_prec`, and
* an optional **product-ion** list from the `Fragments` column matched with `ppm_prod` and an `INTENSITYPERCENT` floor.

Behavior & tips:

* If a row’s `Fragments` is empty, the query is **precursor-only** (still valid for MS2 filtering).
* Fragment values are cleaned (duplicates removed, non-numeric ignored), and zeros are dropped.
* **Cardinality**: by default, requires **1–5** fragment matches (clamped to available fragments). You can override with `cardinality_min` / `cardinality_max`.
* `decimals=4` controls how m/z values are rendered—adjust if your instrument resolution demands more/less precision.
* Tune `ppm_prec` vs `ppm_prod`: product ions often tolerate slightly tighter ppm than precursors, depending on your setup.

The result is written to `MS2_queries_by_compound.txt`, one numbered block per compound.


In [ ]:
# Build
ms2_q = dbs.generate_massql_ms2_queries(
    merged_df2,
    name_col="Compound name",
    mass_col="MolWeight",        # or "Monoisotopic mass"
    fragments_col="Fragments",
    adducts=["[M+H]+","[M+Na]+","[M+K]+","[M+NH4]+", "[M+2H]+2"],
    ppm_prec=10,
    ppm_prod=10,
    cardinality_min= 1,
    cardinality_max= 5,
    intensity_percent=5,
    decimals=4)

# Write
out_path = Path("MS2_queries_by_compound.txt")
with out_path.open("w", encoding="utf-8") as f:
    for i, (name, q) in enumerate(ms2_q.items(), start=1):
        f.write(f"### {i}. {name} ###\n{q}\n\n")

In [ ]:
# Peek a couple:
for i, (k, v) in enumerate(ms2_q.items()):
    print(v, end="\n\n")
    if i == 2:
        break

In [ ]:
STOP

### Prepare a clean submission table for NPClassifier

This cell exports the **minimal structure table** NPClassifier needs: a unique identifier (`InchiKey`) and the corresponding `SMILES`. The file is saved as `<Project>_for_NPClassifier.csv` with no index, ready to upload or batch process.

Tips:

* Consider deduplicating by `InchiKey` first to avoid repeated predictions.
* Make sure all `SMILES` parse in RDKit (invalid rows can be removed earlier).
* If you also want human-readable labels, you can include a `Compound name` column—extra columns are typically ignored by classifiers but remain useful for mapping results back.


In [ ]:
# Constructing the .csv file to be submitted to NPClassifier 
df_NPclassifier0 = merged_df2[["InchiKey","SMILES"]]
df_NPclassifier0.to_csv(Project + '_for_NPClassifier.csv', index = False)

### Attach RDKit molecule objects (and fingerprints) to your table

This cell converts each `SMILES` into an RDKit **Molecule** and adds it as a new `Molecule` column (with embedded fingerprints because `includeFingerprints=True`). Many downstream ops (fingerprints, similarity, drawing) expect this column.

Notes & tips:

* This operation **modifies `merged_df2` in place**.
* It can be **memory-heavy** on large tables (molecules + fingerprints stored per row).
* Your comment mentions `SMILES_parent`, but the code uses `SMILES`. If the source column is actually `SMILES_parent`, change the call to:

  ```python
  PandasTools.AddMoleculeColumnToFrame(merged_df2, 'SMILES_parent', 'Molecule', includeFingerprints=True)
  ```
* If you only need molecules (no fingerprints), set `includeFingerprints=False` to save memory.


In [ ]:
%%capture --no-display
# create the column 'Molecules' with the structures for each SMILES entree at the column 'SMILES_parent'
PandasTools.AddMoleculeColumnToFrame(merged_df2,'SMILES','Molecule',includeFingerprints=True)
print([str(x) for x in  merged_df2.columns])

### Compute an all-vs-all molecular similarity matrix (Morgan fingerprints)

This cell builds a **square similarity matrix** between every pair of structures using **Morgan fingerprints** (radius = 2, nBits = 2048 by default) and the **Dice** metric. Requirements:

* `merged_df2` must already contain an RDKit **`Molecule`** column and a unique **`InchiKey`** per row.
* The output `sim_df` has rows/columns indexed by `InchiKey`, with a **diagonal of 1.0**.

How to read/use it:

* Each entry ∈ \[0, 1] is a pairwise similarity; higher means more similar.
* You can switch to **Tanimoto** by passing `metric="tanimoto"`.
* Tune structure granularity with `radius` (2–3 are common) and control hash space via `nbits` (default 2048 in the helper).

Tips:

* Consider **deduplicating** molecules before running to avoid redundant rows.
* For large libraries, this is **O(n²)** in time/memory; filter first or compute similarities to a **subset/query set** if needed.


In [ ]:
# Assuming merged_df already has RDKit Mol objects in 'Molecule' and an 'InchiKey' column:
sim_df = dbs.morgan_similarity_matrix(
                      merged_df2, 
                      mol_col="Molecule", 
                      id_col="InchiKey", 
                      radius=2, 
                      metric="dice")

display(sim_df.head(2))
print(f"SimTablet2 is the square matrix (with a diagonal = 1, {sim_df.shape}) with all the similarities calculated between every pair of structures.")

### Build a similarity network, export edge list, and list isolated compounds

This cell turns the all-vs-all similarity matrix into a **network**:

* **Edges**: pairs with similarity **> 0.85** (strictly greater; equals are excluded by design).

  * Output: `links_filtered` (columns: `SOURCE`, `TARGET`, `CORRELATION`), deduplicated and undirected.
  * Saved as: `"<Project>_DB_compounds_Similarity_0.85.csv"` (semicolon-separated) if `save_csv=True`.
* **Graph**: `G` is a NetworkX graph built from the edge list; use it for quick stats or to write GraphML/GEXF for Cytoscape/Gephi.
* **Isolated nodes**: entries with **no edges above threshold** (i.e., chemically unique under current settings).

  * Output: `isolated_nodes_df`, saved to `"<Project>_isolated_nodes.csv"` when `save_isolated_csv=True`.

Notes & tips:

* If you want the identity column to be human-readable (e.g., `Compound name` or `file_path`), set `identity_col` accordingly. Right now it’s `InchiKey`.
* Make sure `metadata_df` points to the table that **contains** your identity column. If you annotated into `merged_df2`, pass `metadata_df=merged_df2`.
* To **include** pairs with similarity exactly equal to the cutoff, either slightly lower `threshold` (e.g., `0.8499`) or change the helper to use `>=`.
* Cytoscape import: load the edge CSV; choose `SOURCE`/`TARGET` as source/target and use `CORRELATION` as an edge attribute for styling or filtering.


In [ ]:
links_filtered, G, isolated_nodes_df = dbs.build_similarity_network(
            sim_df,
            threshold=0.85,
            project=Project,
            save_csv=True,
            csv_sep=";",
            metadata_df=merged_df,       # or df_target if that’s where your identities live
            id_col="InchiKey",
            identity_col="InchiKey",    # change if your identity column is different
            save_isolated_csv=True)

# If you want to show InchiKey + file_path like you did:
cols = [c for c in ["InchiKey", "InchiKey"] if c in isolated_nodes_df.columns]
display(isolated_nodes_df[cols] if cols else isolated_nodes_df.head())

### Export custom MS1 databases for MZmine (POS & NEG modes)

This cell creates two **custom MS1 database** CSVs compatible with MZmine:

* **POS** mode using the `[M+H]+` column (`MolWeight+1H`)
* **NEG** mode using the `[M−H]−` column (`MolWeight-1H`)

Each output has the columns (and order) MZmine expects:
`ID, m/z, Retention Time, identity, Formula`

What to know:

* `identity_col` should be a stable identifier you’ll recognize inside MZmine (here we use `InchiKey`; you could use `Compound name` or a filename path).
* `default_rt=0.0` sets a single retention time for all entries (fine for MS1 dereplication lists).
* Filenames default to `"<Project>_MZMine_CustomDB_POS.csv"` and `"<Project>_MZMine_CustomDB_NEG.csv"` in your working folder.

Tips:

* Ensure `MolWeight+1H` and `MolWeight-1H` already exist (use the annotator cell). These typically use the **proton mass** (1.007276466), which is standard for MS.
* You can refine later: add experiment-specific RTs per compound, or create separate DB files per subclass/taxon to speed matching.


In [ ]:
# POS mode: [M+H]+
df_mzmine_pos = dbs.make_mzmine_custom_db(
        merged_df2,
        mz_col="MolWeight+1H",
        formula_col="MolFormula",
        identity_col="InchiKey",   # <— use a column that exists
        default_rt=0.0,
        project=Project,
        mode="POS",
        save_csv=True,)

# NEG mode: [M-H]-
df_mzmine_neg = dbs.make_mzmine_custom_db(
        merged_df2,
        mz_col="MolWeight-1H",
        formula_col="MolFormula",
        identity_col="InchiKey",
        default_rt=0.0,
        project=Project,
        mode="NEG",
        save_csv=True)

### Compute Mordred molecular descriptors (2D/3D) and save to CSV

This cell generates a wide set of **Mordred descriptors** for each SMILES and writes them to `"<Project>_DB_compounds_MordredDescriptors.csv"` (semicolon-separated). The return value `info` reports how many **2D**, **3D**, and **total** descriptors are available in your environment.

Notes & tips:

* `ignore_3D=False` requests **3D descriptors**; unless your molecules already have embedded 3D coordinates, many 3D fields may be **NaN**.

  * If you don’t need 3D (or don’t have conformers), set `ignore_3D=True` for a faster, denser table.
* If you *do* want 3D, generate conformers first (RDKit: `AllChem.EmbedMolecule` + `AllChem.UFFOptimizeMolecule`) and pass those mols to the function.
* Descriptor tables can be **large** (hundreds to thousands of columns). Plan memory accordingly.
* Some Mordred/RDKit builds prefer **NumPy < 2**; if you hit import/runtime issues, pin the version.

In [ ]:
# MORDRED-Descriptors
df_descriptors, info = dbs.compute_mordred_descriptors(
        merged_df2,
        smiles_col="SMILES",
        ignore_3D=False,         # set True if you want 2D-only
        project=Project,         # used for filename
        save_csv=True)

### Cluster compounds with a Ward dendrogram, save the figure, and subset by cluster

This cell performs **hierarchical clustering** on the Mordred descriptor matrix and draws a **Ward dendrogram**. The `color_threshold` both colors branch groups and—since `cut_distance=None`—acts as the **distance cut** to assign cluster IDs. With `run_agglomerative=True`, it also runs a matching **AgglomerativeClustering** to provide a complementary label set.

Outputs:

* **Figure** saved to `images/dendogram.png` (PNG) and displayed inline.
* **CSV** of cluster labels saved with a project-prefixed name.
* **`clusters`**: a `pd.Series` of hierarchical cluster IDs (you attach it to `merged_df2`).
* Example subset: all rows with `cluster == 3`.

Tips:

* Ensure `df_descriptors` is the numeric descriptor table produced earlier (rows aligned to your metadata).
* If you intended the standard spelling, change `filename="dendrogram.png"` to avoid confusion later.
* To control the number of clusters directly, lower `color_threshold` (tighter cut) or pass an explicit `cut_distance`.


In [ ]:
clusters, Z, fig, ax, df_clustered = dbs.dendrogram_and_cluster_descriptors(
        df_descriptors,                    # table from Mordred
        color_threshold=250000,
        xlim=(0, 700000),
        do_save_png=True,
        save_dir="images",
        filename="dendogram.png",
        cut_distance=None,                 # None -> uses color_threshold
        run_agglomerative=True,
        metadata_df=merged_df,             # to fetch file_path (if present)
        id_col="InchiKey",
        identity_col="file_path",
        project=Project,
        save_cluster_csv=True)
fig.show()

# If you want the same “cluster = 3” subset:
merged_df2["cluster"] = clusters  # if not already added via write-back
merged_dfX = merged_df2.loc[merged_df2["cluster"] == 3, ["cluster"]].dropna()

### Project the descriptor space to 2D with t-SNE and explore clusters interactively

This cell uses **t-SNE** to compress the high-dimensional Mordred descriptor space into **2D** for visual exploration. Points are **colored by the Agglomerative cluster labels** (`df_clustered["Clust"]`) and enriched with **hover info** (`file_path`, `cluster`, `Compound name`). The figure renders inline and is also saved as an interactive HTML at `images/t-sne.html` for sharing.

Tips:

* **Perplexity** must be sensible relative to sample size (rule of thumb: ≤ (n−1)/3). For small datasets, use 5–15; for larger, try 20–50.
* Set `standardize=True` if descriptor scales differ markedly.
* Results vary with initialization; add `random_state=0` for reproducibility if desired.
* If colors appear mismatched, ensure the **index** of `df_clustered` aligns with `df_descriptors`.


In [ ]:
# df_descriptors_values from the Mordred step (numeric only)
proj, fig = dbs.tsne_projection_plot(
        df_descriptors,                        # full DF; function picks numeric cols
        metadata_df=merged_df2,                 # for hover columns
        cluster_series=df_clustered["Clust"],  # color by sklearn cluster labels
        hover_cols=("file_path", "cluster", "Compound name"),
        standardize=False,                     # set True if scales differ a lot
        perplexity=20,
        out_html="images/t-sne.html"
)
fig.show()

In [ ]:
merged_df2.columns